We illustrate passing a callback to a function. Note that this function is blocking, and best practice is to fork it into its own thread of execution.

In [1]:
from time import sleep

def slow_calculation():
    res = 0
    for i in range(5):
        res += i * i
        sleep(1)
    return res


Without callbacks

In [2]:
slow_calculation()


30

With a callback that shows intermediate progress.

In [3]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        res += i * i
        sleep(1)
        if cb: cb(i)
    return res


In [4]:
def show_progress(epoch):
    print(f"Awesome! We've finished epoch {epoch}!")


In [5]:
slow_calculation(show_progress)


Awesome! We've finished epoch 0!
Awesome! We've finished epoch 1!
Awesome! We've finished epoch 2!
Awesome! We've finished epoch 3!
Awesome! We've finished epoch 4!


30

We now demonstrate using a lambda callback.

In [6]:
slow_calculation(lambda o: print(f"Awesome! We've finished epoch {o}!"))


Awesome! We've finished epoch 0!
Awesome! We've finished epoch 1!
Awesome! We've finished epoch 2!
Awesome! We've finished epoch 3!
Awesome! We've finished epoch 4!


30

In [7]:
def show_progress(exclamation, epoch):
    print(f"{exclamation}! We've finished epoch {epoch}!")


In [8]:
slow_calculation(lambda o: show_progress("OK I guess", o))


OK I guess! We've finished epoch 0!
OK I guess! We've finished epoch 1!
OK I guess! We've finished epoch 2!
OK I guess! We've finished epoch 3!
OK I guess! We've finished epoch 4!


30

We now demonstrate passing a callback factory as a parameter.

In [9]:
def make_show_progress(exclamation):
    def _inner(epoch):
        print(f"{exclamation}! We've finished epoch {epoch}!")
    return _inner


In [10]:
slow_calculation(make_show_progress("Nice!"))


Nice!! We've finished epoch 0!
Nice!! We've finished epoch 1!
Nice!! We've finished epoch 2!
Nice!! We've finished epoch 3!
Nice!! We've finished epoch 4!


30

We now demonstrate using `partial` to partially supply some arguments to a function, leaving the rest to be supplied at a later time (here, by the function invoking the callback).

In [11]:
from functools import partial

slow_calculation(partial(show_progress, "OK I guess"))


OK I guess! We've finished epoch 0!
OK I guess! We've finished epoch 1!
OK I guess! We've finished epoch 2!
OK I guess! We've finished epoch 3!
OK I guess! We've finished epoch 4!


30

We now demonstrate classes defining callable objects, and use them as callbacks.

In [12]:
class ProgressShowingCallback():
    def __init__(self, exclamation="Awesome"):
        self.exclamation = exclamation

    def __call__(self, epoch):
        print(f"{self.exclamation}! We've finished epoch {epoch}!")


In [13]:
cb = ProgressShowingCallback("Just super")


In [14]:
slow_calculation(cb)


Just super! We've finished epoch 0!
Just super! We've finished epoch 1!
Just super! We've finished epoch 2!
Just super! We've finished epoch 3!
Just super! We've finished epoch 4!


30

We now illustrate the use of `*args` and `**kwargs`

In [15]:
def f(*a, **b):
    print(f"args: {a}; kwargs: {b}")


In [16]:
f(3, 'a', thing1="hello")


args: (3, 'a'); kwargs: {'thing1': 'hello'}


In [17]:
def g(a,b,c=0):
    print(a, b, c)


We demonstrate spreading arguments

In [18]:
args = [1, 2]
kwargs = {'c': 3}
g(*args, **kwargs)


1 2 3


We demonstrate supplying a callback object containing methods conforming to some API, such that the methods will be invoked at the appropriate event/time.

In [19]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb:
            cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb:
            cb.after_calc(i, val=res)
    return res


In [20]:
class PrintStepCallback():
    def before_calc(self, *args, **kwargs):
        print(f"About to start")
    def after_calc (self, *args, **kwargs):
        print(f"Done step")


In [21]:
slow_calculation(PrintStepCallback())


About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step


30

In [22]:
class PrintStatusCallback():
    def __init__(self):
        pass
    def before_calc(self, epoch, **kwargs):
        print(f"About to start: {epoch}")
    def after_calc (self, epoch, val, **kwargs):
        print(f"After {epoch}: {val}")


In [23]:
slow_calculation(PrintStatusCallback())


About to start: 0
After 0: 0
About to start: 1
After 1: 1
About to start: 2
After 2: 5
About to start: 3
After 3: 14
About to start: 4
After 4: 30


30

We now demonstrate supplying a callback object that returns a result that will determine the future behavior of the invoking function. Note also the `hasattr` check, which makes it not an error for a callback to not implement the method in question, in which case it is treated as a no-op.


In [24]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb and hasattr(cb, 'before_calc'):
            cb.before_calc(i)
        res += i * i
        sleep(1)
        if cb and hasattr(cb, 'after_calc'):
            if cb.after_calc(i, res):
                print("stopping early")
                break
    return res


In [25]:
class PrintAfterCallback():
    def after_calc (self, epoch, val):
        print(f"After {epoch}: {val}")
        if val > 10:
            return True


In [26]:
slow_calculation(PrintAfterCallback())


After 0: 0
After 1: 1
After 2: 5
After 3: 14
stopping early


14

We now demonstrate an invoking object that is supplied a callback at time of instantiation.

In [27]:
class SlowCalculator():
    def __init__(self, cb=None):
        self.cb, self.res = cb, 0
    
    def callback(self, cb_name, *args):
        if not self.cb:
            return
        cb = getattr(self.cb, cb_name, None)
        if cb:
            return cb(self, *args)

    def calc(self):
        for i in range(5):
            self.callback('before_calc', i)
            self.res += i * i
            sleep(1)
            if self.callback('after_calc', i):
                print("stopping early")
                break


In [28]:
class ModifyingCallback():
    def after_calc (self, calc, epoch):
        print(f"After {epoch}: {calc.res}")
        if calc.res > 10:
            return True
        if calc.res < 3:
            calc.res = calc.res*2


In [29]:
calculator = SlowCalculator(ModifyingCallback())


In [30]:
calculator.calc()
calculator.res


After 0: 0
After 1: 1
After 2: 6
After 3: 15
stopping early


15